In [2]:
import os
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import f1_score, classification_report, confusion_matrix
from sklearn.utils import resample
from collections import Counter

# ===========================================================
# Step 1. 数据加载
# ===========================================================
data_path = "data"
X = np.load(os.path.join(data_path, "X.npy"))
y = np.load(os.path.join(data_path, "y.npy"))
print(f"原始 X shape: {X.shape} y shape: {y.shape}")
print("原始类别分布:", Counter(y))

# ===========================================================
# Step 2. 过滤坏样本
# ===========================================================
def filter_invalid_samples(X, y, max_abs=10000.0, min_std=1e-6, log_path="data/bad_samples.txt"):
    valid_idx, bad_info = [], []
    for i in range(len(X)):
        xi = X[i]
        reason = None
        if np.isnan(xi).any() or np.isinf(xi).any():
            reason = "NaN/Inf"
        elif np.allclose(xi, 0):
            reason = "All zeros"
        elif np.std(xi) < min_std:
            reason = f"Low std ({np.std(xi):.2e})"
        elif np.any(np.abs(xi) > max_abs):
            reason = f"Out of range (max={np.max(np.abs(xi)):.1f})"

        if reason:
            bad_info.append(f"Sample {i}: {reason}")
        else:
            valid_idx.append(i)

    with open(log_path, "w", encoding="utf-8") as f:
        if bad_info:
            f.write("\n".join(bad_info))
        else:
            f.write("✅ 无坏样本\n")
    print(f"检测并移除 {len(bad_info)} 个坏样本（剩余 {len(valid_idx)} 个），日志保存至 {log_path}")
    return X[valid_idx], y[valid_idx]

X, y = filter_invalid_samples(X, y)

# ===========================================================
# Step 3. 筛选目标标签
# ===========================================================
target_labels = ["p_sit", "p_stand", "a_walk", "p_lie", "t_bend"]
mask = np.isin(y, target_labels)
X, y = X[mask], y[mask]
print(f"过滤后 X shape: {X.shape} y shape: {y.shape}")

# ===========================================================
# Step 4. 特征展开
# ===========================================================
N, T, J, C = X.shape
X = X.reshape(N, T, J * C)
print(f"特征展开后: {X.shape}")

# ===========================================================
# Step 5. 标签编码 + 过采样
# ===========================================================
le = LabelEncoder()
y = le.fit_transform(y)
print("类别映射:", dict(zip(le.classes_, range(len(le.classes_)))))

X_bal, y_bal = [], []
for cls in np.unique(y):
    X_cls = X[y == cls]
    y_cls = y[y == cls]
    X_up, y_up = resample(X_cls, y_cls, replace=True, n_samples=max(Counter(y).values()), random_state=42)
    X_bal.append(X_up)
    y_bal.append(y_up)
X_res = np.vstack(X_bal)
y_res = np.hstack(y_bal)
print("过采样后类别分布:", Counter(y_res))

# ===========================================================
# Step 6. Train / Val / Test 划分
# ===========================================================
X_temp, X_test, y_temp, y_test = train_test_split(X_res, y_res, test_size=0.2, random_state=42, stratify=y_res)
X_train, X_val, y_train, y_val = train_test_split(X_temp, y_temp, test_size=0.25, random_state=42, stratify=y_temp)
print(f"Train: {X_train.shape} Val: {X_val.shape} Test: {X_test.shape}")

X_train, X_val, X_test = map(lambda x: torch.tensor(x, dtype=torch.float32), [X_train, X_val, X_test])
y_train, y_val, y_test = map(lambda y: torch.tensor(y, dtype=torch.long), [y_train, y_val, y_test])

train_loader = DataLoader(TensorDataset(X_train, y_train), batch_size=16, shuffle=True)
val_loader = DataLoader(TensorDataset(X_val, y_val), batch_size=16, shuffle=False)
test_loader = DataLoader(TensorDataset(X_test, y_test), batch_size=16, shuffle=False)

# ===========================================================
# Step 7. ST-GCN 模型定义（修复版）
# ===========================================================
class STGCNBlock(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size_t=3, dropout=0.3):
        super().__init__()
        self.temporal_conv = nn.Conv2d(in_channels, out_channels, kernel_size=(kernel_size_t, 1), padding=(1,0))
        self.relu = nn.ReLU()
        self.bn = nn.BatchNorm2d(out_channels)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        # x: [N, C, T, V]
        x = self.temporal_conv(x)
        x = self.relu(self.bn(x))
        x = self.dropout(x)
        return x

class STGCN(nn.Module):
    def __init__(self, in_channels=3, num_classes=5, dropout=0.3):
        super().__init__()
        self.gcn1 = STGCNBlock(in_channels, 64)
        self.gcn2 = STGCNBlock(64, 128)
        self.gcn3 = STGCNBlock(128, 256)
        self.fc = nn.Linear(256, num_classes)

    def forward(self, x):
        # 输入 x: [N, T, (J*C)]
        N, T, D = x.shape
        J = D // 3
        x = x.view(N, T, J, 3).permute(0, 3, 1, 2)  # -> [N, C, T, J]
        x = self.gcn1(x)
        x = self.gcn2(x)
        x = self.gcn3(x)
        x = x.mean(dim=[2, 3])  # 全局平均池化
        return self.fc(x)

# ===========================================================
# Step 8. 训练与验证 (F1-based Early Stopping)
# ===========================================================
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = STGCN(in_channels=3, num_classes=len(le.classes_)).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.5)

best_f1, patience, wait = 0.0, 8, 0

for epoch in range(1, 41):
    # ---------- Train ----------
    model.train()
    y_true, y_pred, train_loss = [], [], 0
    for xb, yb in train_loader:
        xb, yb = xb.to(device), yb.to(device)
        optimizer.zero_grad()
        out = model(xb)
        loss = criterion(out, yb)
        loss.backward()
        optimizer.step()
        train_loss += loss.item() * xb.size(0)
        y_true.extend(yb.cpu().numpy())
        y_pred.extend(out.argmax(1).cpu().numpy())
    train_f1 = f1_score(y_true, y_pred, average='macro')

    # ---------- Validation ----------
    model.eval()
    y_val_true, y_val_pred = [], []
    with torch.no_grad():
        for xb, yb in val_loader:
            xb, yb = xb.to(device), yb.to(device)
            out = model(xb)
            y_val_true.extend(yb.cpu().numpy())
            y_val_pred.extend(out.argmax(1).cpu().numpy())
    val_f1 = f1_score(y_val_true, y_val_pred, average='macro')

    # ---------- Test ----------
    y_test_true, y_test_pred = [], []
    with torch.no_grad():
        for xb, yb in test_loader:
            xb, yb = xb.to(device), yb.to(device)
            out = model(xb)
            y_test_true.extend(yb.cpu().numpy())
            y_test_pred.extend(out.argmax(1).cpu().numpy())
    test_f1 = f1_score(y_test_true, y_test_pred, average='macro')

    print(f"Epoch {epoch:02d}/40 | Train F1: {train_f1:.4f} | Val F1: {val_f1:.4f} | Test F1: {test_f1:.4f}")

    scheduler.step()

    if val_f1 > best_f1:
        best_f1, wait = val_f1, 0
        torch.save(model.state_dict(), "data/stgcn_pose5_fixed.pth")
    else:
        wait += 1
        if wait >= patience:
            print("⚠️ 提前停止：验证集 F1 未提升。")
            break

print(f"✅ 最佳验证 F1: {best_f1:.4f}")

# ===========================================================
# Step 9. 测试报告输出
# ===========================================================
model.load_state_dict(torch.load("data/stgcn_pose5_fixed.pth", weights_only=True))
model.eval()
y_true, y_pred = [], []
with torch.no_grad():
    for xb, yb in test_loader:
        xb, yb = xb.to(device), yb.to(device)
        out = model(xb)
        y_true.extend(yb.cpu().numpy())
        y_pred.extend(out.argmax(1).cpu().numpy())

print("\n🎯 最终测试集性能:")
print(classification_report(y_true, y_pred, target_names=le.classes_, zero_division=0))
print("混淆矩阵:")
print(confusion_matrix(y_true, y_pred))


原始 X shape: (144, 100, 32, 3) y shape: (144,)
原始类别分布: Counter({np.str_('p_stand'): 45, np.str_('a_walk'): 28, np.str_('p_sit'): 27, np.str_('Unknown'): 17, np.str_('t_bend'): 7, np.str_('p_lie'): 6, np.str_('p_bent'): 6, np.str_('t_straighten'): 4, np.str_('t_bed_turn'): 3, np.str_('t_stand_to_sit'): 1})
检测并移除 0 个坏样本（剩余 144 个），日志保存至 data/bad_samples.txt
过滤后 X shape: (113, 100, 32, 3) y shape: (113,)
特征展开后: (113, 100, 96)
类别映射: {np.str_('a_walk'): 0, np.str_('p_lie'): 1, np.str_('p_sit'): 2, np.str_('p_stand'): 3, np.str_('t_bend'): 4}
过采样后类别分布: Counter({np.int64(0): 45, np.int64(1): 45, np.int64(2): 45, np.int64(3): 45, np.int64(4): 45})
Train: (135, 100, 96) Val: (45, 100, 96) Test: (45, 100, 96)
Epoch 01/40 | Train F1: 0.3577 | Val F1: 0.1926 | Test F1: 0.1608
Epoch 02/40 | Train F1: 0.4798 | Val F1: 0.4547 | Test F1: 0.3952
Epoch 03/40 | Train F1: 0.4853 | Val F1: 0.5695 | Test F1: 0.4403
Epoch 04/40 | Train F1: 0.5429 | Val F1: 0.5293 | Test F1: 0.5665
Epoch 05/40 | Train F1: 0.550